## **Preface**
This guide aims to extend the previous guide to learn more in depth about building assets.

## **Prerequisites**
Python examples in this guide requires only having `usd-core` library
```
pip install usd-core
```

## **Better Asset Structure**

Let's examine an existent solution for creating USD assets.

[Houdini Component Builder](https://www.sidefx.com/docs/houdini/solaris/component_builder.html) is a tool developed by sidefx that automates or enforces various best practices (this is especially useful for artists and studios new to USD).

Acknowledgement, Many thank you to [USD and LOPs for Houdini Artists | FXPHD](https://www.fxphd.com/details/642/) for the amazing tips about better asset structures.


> I'll modify the layer structure generated by component builder a little bit.
>
> For sake of simplicity, I'll use the default purpose. but, I'll include code for creating prims with proxy and render purposes.

### **Geo Layer**
Layer Structure: 
```
geo.usda
  ├── mata data
  |     └── default prim: {asset-name}
  ├── over ASSET_{variant-set-name}_variant_{variant-name}
  |          ├── mata data
  |          |     ├── {variant-set-name}
  |          |     └──  hidden = true
  |          └── xForm 'ASSET'
  |                └── {variant-set-name}
  |                        └── {variant-name}
  |                               ├── meta data
  |                               |    └── Kind: component 
  |                               └── Scope Geo
  |                                    ├── extentsHint
  |                                    ├── Scope proxy
  |                                    |     ├── purpose proxy
  |                                    |     └── geo prim
  |                                    └── Scope render
  |                                          ├── rel: proxy-prim
  |                                          ├── purpose render
  |                                          └── geo prim
  └── def {asset-name}
           └── mata data
                   └── references
                         └── variant_{variant-set-name}_{variant-name}/ASSET
```
Adding new variants is done through repeating the same process for `ASSET_{variant-set-name}_variant_{variant-name}`

In [7]:
# create Geo Layer
from pxr import Usd, Sdf, UsdGeom, Kind

variantSets = {
    'model' : {
                
        'normal' : {'xformOp:scale' : (1, 1, 1),
                    'extentsHint'   : [(-1, -1, -1), (1, 1, 1)]},
    
        'short'  : {'xformOp:scale' : (1, 0.5, 1),
                    'extentsHint'   : [(-1, -0.5, -1), (1, 0.5, 1)]},
       
        'tall'   : {'xformOp:scale' : (1, 2, 1),
                    'extentsHint'   : [(-1, -2, -1), (1, 2, 1)]}
    }
}


attr_types = {
   'extentsHint' :  Sdf.ValueTypeNames.Float3Array,
    'xformOp:scale' : Sdf.ValueTypeNames.Float3
}


stage = Usd.Stage.CreateInMemory()

my_model = stage.DefinePrim('/My_Model')
stage.SetDefaultPrim(my_model.GetPrim())

for variantSetName, variantSet in variantSets.items():
    for variantName, variantData in variantSet.items():

        variant = stage.OverridePrim('/ASSET_{}_variant_{}'.format(variantSetName, variantName))
        variant .SetHidden(1)
        
        asset = UsdGeom.Xform.Define(stage, variant.GetPath().AppendPath('ASSET'))
        
        model_variant_set = asset.GetPrim().GetVariantSets().AddVariantSet(variantSetName)
        model_variant_set.AddVariant(variantName)
        model_variant_set.SetVariantSelection(variantName)
        with model_variant_set.GetVariantEditContext():
            Usd.ModelAPI(asset).SetKind(Kind.Tokens.component)
            geoPrim = UsdGeom.Scope.Define(stage, asset.GetPath().AppendPath('Geo'))
            cube_model = UsdGeom.Cube.Define(stage, geoPrim.GetPath().AppendPath('Cube'))
            
            attrName = 'xformOp:scale'
            cube_model.GetPrim().CreateAttribute(attrName, attr_types[attrName]).Set(variantData[attrName])
            cube_model.GetPrim().CreateAttribute('xformOpOrder', Sdf.ValueTypeNames.Token).Set([attrName])
            attrName = 'extentsHint'
            asset.GetPrim().CreateAttribute(attrName, attr_types[attrName]).Set(variantData[attrName])  # Add extents hint to asset
            
            # proxy_model = UsdGeom.Scope.Define(stage, geoPrim.GetPath().AppendPath('Proxy'))
            # proxy_model.GetPrim().CreateAttribute('purpose', Sdf.ValueTypeNames.Token).Set('proxy')
            # proxy_cube_model = UsdGeom.Cube.Define(stage, proxy_model.GetPath().AppendPath('Cube'))
            
            # render_model = UsdGeom.Scope.Define(stage, geoPrim.GetPath().AppendPath('Render'))
            # render_model.GetPrim().CreateAttribute('purpose', Sdf.ValueTypeNames.Token).Set('render')
            # render_model.GetPrim().CreateRelationship('proxyPrim').SetTargets([proxy_model.GetPath()])
            # render_cube_model = UsdGeom.Cube.Define(stage, render_model.GetPath().AppendPath('Cube'))

        variant.SetSpecifier(Sdf.SpecifierOver)  # force prim to be over.
        my_model.GetReferences().AddInternalReference(asset.GetPath())

print(stage.GetRootLayer().ExportToString())
stage.GetRootLayer().Export('examples/build_assets_with_python_2/better_asset_structure/geo.usda')

#usda 1.0
(
    defaultPrim = "My_Model"
)

def "My_Model" (
    prepend references = [
        </ASSET_model_variant_normal/ASSET>,
        </ASSET_model_variant_short/ASSET>,
        </ASSET_model_variant_tall/ASSET>
    ]
)
{
}

over "ASSET_model_variant_normal" (
    hidden = true
)
{
    def Xform "ASSET" (
        variants = {
            string model = "normal"
        }
        prepend variantSets = "model"
    )
    {
        variantSet "model" = {
            "normal" (
                kind = "component"
            ) {
                custom float3[] extentsHint = [(-1, -1, -1), (1, 1, 1)]

                def Scope "Geo"
                {
                    def Cube "Cube"
                    {
                        custom float3 xformOp:scale = (1, 1, 1)
                        uniform token[] xformOpOrder = ["xformOp:scale"]
                    }
                }

            }
        }
    }
}

over "ASSET_model_variant_short" (
    hidden = true
)
{
    def Xform "

True

## **Mtl Layer**
Layer structure 1: Material libarary contains all materials (e.g. Red, Green and Blue) + variant bindings (Red material binding, ...)

```
mtl.usda
  ├── mata data
  |     └── default prim: {asset-name}
  ├── over ASSET-Material-Library
  |          └── Scope Mtl
  |               └── Material 'variant-material-name'
  |                     ├── output:surface
  |                     └── Shader 'variant-material-name'
  |                           └── {variant-name}
  └──  def {asset-name}
               ├── mata data
               |     ├── references
               |     |      └── ASSET-Material-Library
               |     └── {variant-set-name}
               ├── Scope Mtl (empty prim)
               └── {variant-set-name}
                      └── {variant-material-name}
                            └── over Geo
                               ├── over proxy
                               |     ├── apiSchemas "MaterialBindingAPI"
                               |     └── rel material:binding  {variant-material-name}_Preview 
                               └── over render
                                     ├── apiSchemas "MaterialBindingAPI"
                                     └── rel material:binding  {variant-material-name}
```

Layer structure 2: Variant Material and bindings (e.g. Red Material with its binding, ...) 
In short: `def material + over geo(binding)`
```
mtl.usda
  ├── mata data
  |     └── default prim: {asset-name}
  ├── over ASSET_{variant-set-name}_variant_{material-name}
  |          ├── mata data
  |          |      └──  hidden = true
  |          └── xForm 'ASSET'
  |                  ├── mata data
  |                  |      └── {variant-set-name}
  |                  ├── Scope Mtl
  |                  |      └── Material 'variant-material-name'
  |                  |            ├── output:surface
  |                  |            └── Shader 'variant-material-name'
  |                  |                   └── {variant-name}
  |                  └── {variant-set-name}
  |                         └── {variant-name}
  |                                └── over Geo
  |                                   ├── over proxy
  |                                   |     ├── apiSchemas "MaterialBindingAPI"
  |                                   |     └── rel material:binding  {variant-material-name}_Preview 
  |                                   └── over render
  |                                         ├── apiSchemas "MaterialBindingAPI"
  |                                         └── rel material:binding  {variant-material-name}
  └──  def {asset-name}
             └── mata data
                  └── references
                          └── ASSET_{variant-set-name}_variant_{material-name}
```
Adding new variants is done through repeating the same process for `ASSET_{variant-set-name}_variant_{material-name}`

> I'll use `Layer structure 2` for consistency.
> Also, to avoid loading unused materials.

In [10]:
# Create Material Layer
# Material X won't be used in this example as it's not available inside usd-core.
from pxr import Usd, Sdf, UsdGeom, UsdShade, Kind

variantSets = {
    'material' : {
        'red'   : {'diffuseColor' : (1, 0, 0)},
        'green' : {'diffuseColor' : (0, 1, 0)},
        'blue'  : {'diffuseColor' : (0, 0, 1)},      
    }
}

stage = Usd.Stage.CreateInMemory()

my_model = stage.DefinePrim('/My_Model')
stage.SetDefaultPrim(my_model.GetPrim())

for variantSetName, variantSet in variantSets.items():
    for variantName, variantData in variantSet.items():
        variant = stage.OverridePrim('/ASSET_{}_variant_{}'.format(variantSetName, variantName))
        variant .SetHidden(1)
        
        asset = UsdGeom.Xform.Define(stage, variant.GetPath().AppendPath('ASSET'))
        
        materialVariantSet = asset.GetPrim().GetVariantSets().AddVariantSet(variantSetName)
        materialVariantSet.AddVariant(variantName)
        materialVariantSet.SetVariantSelection(variantName)
        with materialVariantSet.GetVariantEditContext():
        
            materialScope = UsdGeom.Scope.Define(stage, asset.GetPath().AppendPath('Mtl'))
            
            materialVariant = UsdShade.Material.Define(stage, materialScope.GetPath().AppendPath(variantName))
            shaderPreview = UsdShade.Shader.Define(stage, materialVariant.GetPath().AppendPath('{}Shader'.format(variantName)))
            materialVariant.CreateSurfaceOutput().ConnectToSource(shaderPreview.ConnectableAPI(), "surface")
            shaderPreview.CreateIdAttr('UsdPreviewSurface')
            shaderPreview.CreateInput('diffuseColor', Sdf.ValueTypeNames.Color3f).Set(variantData['diffuseColor'])

            cubePrim = stage.OverridePrim(asset.GetPath().AppendPath('Geo/Cube'))  # Same prim name as in geo.usda
            
            cubePrim.GetPrim().ApplyAPI("MaterialBindingAPI")
            UsdShade.MaterialBindingAPI(cubePrim.GetPrim()).Bind(materialVariant)  # add material binding to geo override
        
        variant.SetSpecifier(Sdf.SpecifierOver)  # force prim to be over.
        my_model.GetReferences().AddInternalReference(asset.GetPath())

print(stage.GetRootLayer().ExportToString())
stage.GetRootLayer().Export('examples/build_assets_with_python_2/better_asset_structure/mtl.usda')

#usda 1.0
(
    defaultPrim = "My_Model"
)

def "My_Model" (
    prepend references = [
        </ASSET_material_variant_red/ASSET>,
        </ASSET_material_variant_green/ASSET>,
        </ASSET_material_variant_blue/ASSET>
    ]
)
{
}

over "ASSET_material_variant_red" (
    hidden = true
)
{
    def Xform "ASSET" (
        variants = {
            string material = "red"
        }
        prepend variantSets = "material"
    )
    {
        variantSet "material" = {
            "red" {
                def Scope "Mtl"
                {
                    def Material "red"
                    {
                        token outputs:surface.connect = </ASSET_material_variant_red/ASSET/Mtl/red/redShader.outputs:surface>

                        def Shader "redShader"
                        {
                            uniform token info:id = "UsdPreviewSurface"
                            color3f inputs:diffuseColor = (1, 0, 0)
                            token outputs:surface
     

True

## **Payload Layer**
Layer Structure:
```
payload.usda
  ├── mata data
  |     └── default prim: {asset-name}
  └──  def {asset-name}
            └── meta data
                 ├── Kind: component 
                 └── references
                       ├── geo.usda
                       └── mtl.usda
```

In [13]:
# Create Payload
from pxr import Usd, Kind

stage = Usd.Stage.CreateInMemory()

my_model = stage.DefinePrim('/My_Model')
stage.SetDefaultPrim(my_model.GetPrim())

Usd.ModelAPI(my_model).SetKind(Kind.Tokens.component)

my_model.GetPrim().GetReferences().AddReference('./geo.usda')  # hardcoded layer name
my_model.GetPrim().GetReferences().AddReference('./mtl.usda')  # hardcoded layer name

print(stage.GetRootLayer().ExportToString())
stage.GetRootLayer().Export('examples/build_assets_with_python_2/better_asset_structure/payload.usda')

#usda 1.0
(
    defaultPrim = "My_Model"
)

def "My_Model" (
    kind = "component"
    prepend references = [
        @./geo.usda@,
        @./mtl.usda@
    ]
)
{
}




True

## **Asset Layer**
Layer Structure:
```
my_asset.usda
  ├── mata data
  |     └── default prim: {asset-name}
  ├──  def {asset-name}
  |         ├── mata data
  |         |     ├── asset info
  |         |     |     ├── identifier (asset layer itself)
  |         |     |     ├── thumbnail (a thumbnail for the asset this is added for feature parity with Houdini)
  |         |     |     └── name
  |         |     ├── inherits __class__/{asset-name}
  |         |     ├── payload payload.usda
  |         |     ├── Variant Sets : [{model-variant-set}, {material-variant-set}]
  |         |     └── Kind: component
  |         ├── {model-variant-set}
  |         |     └── {variant-model-name} (empty variants, model variants should include extentHint property)
  |         |          ├── meta data
  |         |          |     ├── apiSchemas "GeomModelAPI"
  |         |          |     └── Kind: component
  |         |          └── extentsHint
  |         └── {material-variant-set}
  |               └── {variant-material-name} (empty variants)
  └── class __class__
          └── {asset-name}
```

In [25]:
# Create Asset Layer
from pxr import Usd, Sdf, UsdGeom, UsdShade

variantSets = {
    'model' : {
        'tall'   : {'xformOp:scale' : (1, 2, 1),
                    'extentsHint'   : [(-1, -2, -1), (1, 2, 1)] },
        
        'short'  : {'xformOp:scale' : (1, 0.5, 1),
                    'extentsHint'   : [(-1, -0.5, -1), (1, 0.5, 1)]},
        
        'normal' : {'xformOp:scale' : (1, 1, 1),
                    'extentsHint'   : [(-1, -1, -1), (1, 1, 1)]}        
    },
    'material' : {
        'blue'  : {'diffuseColor' : (0, 0, 1)}, 
        'green' : {'diffuseColor' : (0, 1, 0)},
        'red'   : {'diffuseColor' : (1, 0, 0)},             
    }
}

attr_types = {
   'extentsHint' :  Sdf.ValueTypeNames.Float3Array,
    'xformOp:scale' : Sdf.ValueTypeNames.Float3
}


stage = Usd.Stage.CreateInMemory()

my_model = stage.DefinePrim('/My_Model', 'Xform')
stage.SetDefaultPrim(my_model.GetPrim())

my_model_class = stage.CreateClassPrim('/__class__')
my_model_class = stage.CreateClassPrim(my_model_class.GetPath().AppendPath('My_Model'))
my_model.GetPrim().GetInherits().AddInherit(my_model_class.GetPath())

my_model.SetAssetInfo( {'identifier' : Sdf.AssetPath('./my_model.usda') , 'name' : 'My_Model'})
my_model.SetAssetInfoByKey('thumbnail', Sdf.AssetPath('thumbnail-path'))  # FYI, If I were inside houdini, I would actually save a view snap
                                                                          # and save it as my thumbnail.
                                                                          # https://www.sidefx.com/docs/houdini/commands/viewsnapshot.html           
my_model.GetPrim().GetPayloads().AddPayload('./payload.usda', '/My_Model')

Usd.ModelAPI(my_model).SetKind(Kind.Tokens.component)

for variantSetName, variantSet in variantSets.items():
    for variantName, variantData in variantSet.items():
        
        variantSet = my_model.GetPrim().GetVariantSets().AddVariantSet(variantSetName)
        variantSet.AddVariant(variantName)
        variantSet.SetVariantSelection(variantName)

        if variantSetName == 'model':
            with variantSet.GetVariantEditContext():
                Usd.ModelAPI(my_model).SetKind(Kind.Tokens.component)
                
                my_model.GetPrim().ApplyAPI("GeomModelAPI")
                
                attrName = 'extentsHint'
                my_model.GetPrim().CreateAttribute(attrName, attr_types[attrName]).Set(variantData[attrName])  # Add extents hint to asset         
    
print(stage.GetRootLayer().ExportToString())
stage.GetRootLayer().Export('examples/build_assets_with_python_2/better_asset_structure/my_model.usda')

#usda 1.0
(
    defaultPrim = "My_Model"
)

def Xform "My_Model" (
    assetInfo = {
        asset identifier = @./my_model.usda@
        string name = "My_Model"
        asset thumbnail = @thumbnail-path@
    }
    prepend inherits = </__class__/My_Model>
    kind = "component"
    prepend payload = @./payload.usda@</My_Model>
    variants = {
        string material = "red"
        string model = "normal"
    }
    prepend variantSets = ["model", "material"]
)
{
    variantSet "model" = {
        "normal" (
            prepend apiSchemas = ["GeomModelAPI"]
            kind = "component"
        ) {
            custom float3[] extentsHint = [(-1, -1, -1), (1, 1, 1)]

        }
        "short" (
            prepend apiSchemas = ["GeomModelAPI"]
            kind = "component"
        ) {
            custom float3[] extentsHint = [(-1, -0.5, -1), (1, 0.5, 1)]

        }
        "tall" (
            prepend apiSchemas = ["GeomModelAPI"]
            kind = "component"
        ) {
         

True

**Result:** To be honset, it looks the same as our previous assets. But, This structure provides more capabilities!
> Notice how `usdview` uses `extentsHint` property to draw the bounding box while pay is unloaded.

> I had to click on the view port in `usdview` to show the updated boudning box value.

![](../resources/build_assets_with_python_2/better_asset_structure.gif)